In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

root_dir = '/content/gdrive/MyDrive/Data'

Mounted at /content/gdrive


In [3]:
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import tensorflow as tf
from tensorflow import keras

In [32]:
from pandas.core.frame import DataFrame
from time import sleep

def PreprocessAWSFile(aws_file, sheet_name):
  aws = pd.read_excel(aws_file, sheet_name=9)
  aws = aws.drop(['Line', 'Epoch', 'Day', 'Seconds', 'Off-Wrist Status', 'Marker', 
                  'White Light', 'Red Light', 'Green Light', 'Blue Light', 
                  'Sleep/Wake', 'Mobility', 'Interval Status', 'S/W Status'], axis=1)

  aws_complete_minute = pd.DataFrame(aws['Activity'])
  aws_complete_minute['Label'] = aws.apply(
      lambda x: str(x['Date'])[:10] + ' ' + x['Time'].strftime("%H:%M:%S"), axis=1)

  aws_complete_half_minute = pd.DataFrame({'Activity': [np.nan] * aws.shape[0]})
  aws_complete_half_minute['Label'] = aws.apply(
      lambda x: str(x['Date'])[:10] + ' ' + x['Time'].strftime("%H:%M")+':30', axis=1)

  aws = pd.concat([aws_complete_half_minute, aws_complete_minute]).sort_values(
      by=['Label']).reset_index(drop=True)

  aws['Activity'] = aws['Activity'].interpolate().replace(np.nan, 0)
  return aws

def PreprocessAX3File(ax3_file, chunks = 2):
  ax3 = pd.read_csv(ax3_file, header=None, squeeze=True)
  section_size = int(ax3.shape[0] / chunks)
  for i in range(chunks):
    ax3_processed_part = PreprocessAX3Data(ax3[i * section_size: section_size * (i + 1)])
    ax3_processed_part.to_csv('part' + str(i+1) + '.csv', index=False)
    ax3_processed_part = None
    print(i+1)
    sleep(40) # Waiting for RAM to get emptied
  ax3 = None

def GetProcessedAX3Data(chunks = 2):
  ax3 = None
  for i in range(chunks):
    file_name = 'part' + str(i+1) + '.csv'
    temp = pd.read_csv(file_name, squeeze=True, index_col=False)
    ax3 = pd.concat([ax3, temp])
    os.remove(file_name)
    temp = 0
  ax3 = ax3.reset_index()
  return ax3

def PreprocessAX3Data(ax3):
  ax3.columns = (['Date_time', 'X', 'Y', 'Z', 'Light', 'Temp'])
  ax3 = ax3.drop(["Light"], axis=1)
  ax3 = ax3[:-1]

  ax3['Label'] = ax3['Date_time'].map(lambda x : 
                x[0:17] + str(int(float(x[17:])>=30)*3) + "0")

  ax3 = ax3.groupby(['Label']).agg({'X': lambda x : list(x), 
                                    'Y': lambda x : list(x), 
                                    'Z': lambda x : list(x), 
                                    'Temp': lambda x : list(x)}).reset_index()
  
  return ax3

Preprocessing AWS Data

In [33]:
aws_file = os.path.join(root_dir, "18 participants/AWS/SDRI001_AWS_ALL_ALLV4_N1.xlsx")
aws = PreprocessAWSFile(aws_file, 0)
print(aws.head(5))

   Activity                Label
0       0.0  2020-02-17 12:54:00
1       0.0  2020-02-17 12:54:30
2       0.0  2020-02-17 12:55:00
3       0.0  2020-02-17 12:55:30
4       0.0  2020-02-17 12:56:00


Preprocessing AX3 Data

In [4]:
ax3_dir = os.path.join(root_dir, "18 participants/AX3/Extracted")
psg_dir = os.path.join(root_dir, "18 participants/PSG")

ax3_files = os.listdir(ax3_dir)
psg_files = os.listdir(psg_dir)

participant = 10

print(ax3_files[0])
ax3_file_part1 = os.path.join(ax3_dir, "AX3_step1_sub17.csv")
#ax3_file_part2 = os.path.join(ax3_dir, "AX3_step1_sub" + str(participant) + "_part2.csv")

#ax3 = None
ax3_part1 = None
#ax3_part2 = None

ax3_part1 = PreprocessAX3File(ax3_file_part1)
#ax3_part2 = PreprocessAX3File(ax3_file_part2)

#ax3 = pd.concat([ax3_part1, ax3_part2]).reset_index()

#print(ax3.columns)
#print(ax3.head(5))
#print(ax3.tail(1))



AX3_step1_sub18.csv
1
2


Merging AX3 and AWS Data

In [23]:
ax3.to_csv("AX3_step1_sub17.csv", index=False)

In [22]:
ax3 = GetProcessedAX3Data(chunks = 2)

print(ax3.columns)


Index(['Label', 'X', 'Y', 'Z', 'Temp'], dtype='object')
